### UC Berkeley, MICS, W202-Cryptography
### Week 02  Breakout 2
### Forging an RSA Digital Signature using the Homomorphic Property of RSA and Preventing Attacks with Padding


RSA is traditionally associated with using asymmetric encryption for securely establishing keys for symmetric encryption / decryption.  In this case <n,e> is the public key, and d is the private key.

RSA can also be used for digital signatures.  In this case, e is the private key and <n,d> is the public key. An entity encrypts a message in plaintext m to produce ciphertext c.  The receiver is given the <n,d> the public key, the message in plaintext m, and the ciphertext c.  The receiver decrypts c and if it matches m, the signature is valid. The entity creating the digital signatures must keep e, p, q secret.

If we discover e, we can of course forge a digital signature.

In usage of digital signatures, e is a private key, if someone uses a "short exponent" (3, 7, 65537) for e, we can easily recover e. Remember, NEVER EVER use a short exponent for a private key - it can easily be recovered!

If we can factor n into p and q, we can recover e and forge a digital signature, using the same method we used to break RSA by factoring.

There is another way to create a forged digital signature without knowing e.  RSA is homomorphic. If we have two valid digital signatures, we can multiply them together and create a forged digital signature, which decrypted will be equal to the product of the two plaintext messages. We do NOT need to know e to do this!

To guard RSA digital signatures against attacks using its homomorphic property, we use padding.  We will try a simple pad of adding a 1 as a new least significant digit in the message and demonstrate that it blocks attacks using the homomorphic property.

However, a simple pad introduces patterns that could possibly be exploited by attackers.

Next we will append a different random integer with the same number of digits as new least significant digits so there will not be a pattern created by the padding in the digital signatures.

These are simple examples to show how the math works.  In the real world, more elaborate padding schemes such as OAEP (Optimal Asymmetric Encryption Padding) are used.

### Forging a Digital Signature using the Homomorphic Property of RSA

In [1]:
from sage.all import *

In [2]:
def my_print_number(label, x):
    "prints a number in decimal, number of digits, hex, number of bits"
    
    print ("\n", label, '\n')
    print ("decimal:", "{:,}".format(x), "\n")
    print ("number of digits:", x.ndigits(), "\n")
    print ("hex:", x.hex(), "\n")
    print ("number of bits:", x.nbits(), "\n")

In [3]:
# we will use 4096 bits for n

# <n,d> is the public key

n = 0x7e4aa443048ed9250fc4ccd5c902eaac304b0d72f2f74b11b0febba1f09c8839e7733028352a0b7a132cc4e955159d9c6c8fe573ab3e1f7ed4552925b404b34d9a0dd394e57b8fe4427c6a23c9c8cebeb74b9a87719fb910d6683e2138d110301d1ea9fcb7e1d72f0384065bdd16005e03358a2125f1d28579c72a0c186b9f2b27b5dd4409ff1b50f06e689b2b40c239af7ce60610348e073f3dbefc021543be7a6f23be43403972cce0534388a9494ca4e71568796140a9ad21459c16b02006d3701207ece28ff71f132c11ee4fa7fd9b3d4c991f91a176d062631e0a7e52507b58d57bd7fba361ad639d46931a8e6b4cb4d57c94e72b82b691f52035a06d33d3d023c7c534eb56d83a89ce491587c204aea52de7b82fe2803c5db232eace02dd46f81a4e7ac1af14d74b54dcf990d7ec596369e6803732510c32194617b9e63392afa006fe69c12b4244a88004bab4fc084c98ff62f581dbd575b0b093d63f883b36fa59a04f1d6f51454b2298c368fc6e73b8a835b852e0cf4e7bbad1ccccf1fafd2f374860e7c361b852bb0db099359bd5e56cff6693714dbaa45e46cf524e428eb57a42762c386cbd808a3b041b3500596ec22562f662242a790efa73590fa1013baaba1b39e1d51d4bda8028f1ec719d0126803296f109467b1a378348e5787cfac4616628d295b480b566cb58573d58364b8a04a20d00686cb99620f1

d = 0x658e2d542331e188c13be1c06d48c4ba3757642dd3ccb67feada2e2b40a9fdc6fb7097c8af6139cbdaf9e67b1552805c7478d78e5a4c16e864ebf1aee3b8547f0499985c9f5b577c00afb6d74ca143310b3081cab868d14ae7b4491cb5a429bc69172eaabda673837286c0143d3fdb7bfdf6d3792449604e8cf18cc207235221ef15249b3949da3f6222abafa05f0bf8f6f42c1ceb56e1901af85d0ffb1a6ac8cebc879f6ca569dd85fb2488b4e6c8580ee1d14722c360c685a442b84ca730f2eca2b4f3fff55c2323327ba656c691d9886ebbdef7710b41fd38b5df3d6214e5530ce537347ca943946179fa53023b11e6e211de76686be428be4fd5afcb3948ac530127a8d262e64910162bc3a745320608f0ea8666e44a7cc68dd473c3fc83565d95df2e22280db9c9e20aee4c1d125bf633096c4c9def5f51544394e96cc44aaddd39ac6b510eb6ab15a96cf3eed9cc7a472e613416f2a4f2e1f8d15d0d683b34d6d8629cb07a402780ac03d90cea22136998067ce1a2a624c221df73244715754692054c0abf5ae1ae92878deee46d6ee8f0107f021efa324d2920d7f624d857237aeb755aaaaa82cb1d346e3b835da41af1fe8e1efe405dd6f74df17adf7d45b1352cedc74fdfe2bc0eab9472f1f4ad9f21b5ff4a2e79fa2ece46e31b06c5dba552bc258feac1a24f95a04f898a081af2e758183d269c7c7d5f1b3bfa09


In [4]:
my_print_number("n", n)


 n 

decimal: 515,224,650,050,620,411,306,086,851,673,193,148,477,154,601,140,962,241,246,547,244,037,527,913,691,910,145,038,365,745,750,200,630,141,158,536,334,275,583,438,458,835,100,114,638,937,637,504,607,752,689,182,441,445,137,444,248,099,977,101,604,831,084,295,024,311,810,899,429,416,906,062,135,471,637,477,932,943,139,824,140,830,567,313,870,350,594,191,262,398,740,736,780,102,719,105,228,928,909,231,566,543,406,157,053,326,149,522,547,758,325,836,547,935,984,249,237,367,668,553,947,163,773,420,007,870,151,037,192,491,801,233,923,296,359,015,100,276,258,164,275,363,889,971,328,821,052,734,754,860,141,298,914,521,668,235,552,449,717,497,283,534,991,526,741,483,056,860,379,710,343,404,857,858,926,635,533,626,681,430,926,212,163,639,525,174,065,827,275,399,645,281,603,356,565,219,066,312,395,839,358,488,327,813,683,594,848,185,618,375,986,483,080,594,645,680,775,898,243,902,996,899,864,068,879,330,848,820,304,201,220,984,573,189,841,408,402,805,163,752,785,391,352,224,201,264,8

In [5]:
my_print_number("d", d)


 d 

decimal: 414,309,800,167,637,694,275,984,097,750,325,653,689,104,102,797,857,047,446,398,070,267,528,781,385,661,137,980,024,355,946,319,581,237,523,504,426,194,962,804,524,377,897,490,081,150,487,289,179,130,000,142,899,907,222,010,317,294,599,604,443,377,647,997,210,783,224,263,636,129,031,192,199,922,333,784,764,459,258,429,231,938,950,956,808,681,094,163,673,108,450,162,030,083,461,139,665,124,871,582,967,137,158,636,470,321,768,987,636,207,373,558,358,266,232,625,842,185,538,847,843,367,808,292,171,204,793,129,915,650,426,498,488,660,035,761,269,686,859,654,464,237,903,908,485,782,976,322,406,075,381,191,527,188,816,277,920,172,311,681,437,978,708,148,199,519,134,237,068,157,496,049,248,055,981,326,771,458,161,081,105,756,310,872,838,198,021,358,478,663,777,029,264,239,514,560,606,605,112,387,299,732,718,216,431,254,018,509,670,119,198,270,961,693,545,711,936,765,989,193,390,427,995,704,123,079,803,681,483,670,486,630,597,054,136,270,962,011,401,446,956,894,854,375,543,428,3

In [6]:
# we will use e to create the digital signatures.  

# e is the secret private key.

e = 0x47e14f1b7a805d85f2c2be44edaccfd7a041bd33a8a76ebaf3befabc6d9cb95266e8cfd51979e74e50b58c183215934563ad3abf4e9cfdf03dade4bc74c6e41c991725f41e6d826f2c256745f4015917139db2916ce5f3ef5e675c69324a8d53396a9a0aa35ebdd290fa032348ae14ea7c909e4ae86d0b970719833e4ac889727f9ca2385fc063538bbed6a114cbfd4ee57c268536a5ee1446f82cfdf291e783f695d6fbfed7c626ee4ba484c20d63d13011c645bb1d1490aae596226d97265971c4f7f75401f40b30e63f98ef233c72481414beb7c0989cd98da8fb77086d9a1f856e2a44208d8fa66e8cfd469aa8aa53b11cff52460ac25706cdb168f9ac057d96a272a9df082266d943d141efcada0f1d2671af00086606c784610b887fc1c7fe6dbe779ad99566c1aa487b6e89e6dcab266bbd286c16ca4b11873505c4637a63cc99b0bc88e28d24e86783a987746cbc69db68b89c581ed50c4bc47b256c1d5b41c153820e7a19bba8f0e1e80b1bbf28593bb97ded479d72ecd1a1c1a99d7faaaf702d4570d553a5db8a13b738b40012740110a2dd7abd2677d9a8a67efb2cbe9a8b08499f59d99902bb64e5b163533f6f48ea29b2f3526e55b5ad36b221d0bc4f4c9c34f198a2de58c0a4eedd1847640d966957254c1ed6e5016c86ef7501b397bbf1380f57bfdc9aadd3071f282f8a61990840c06e2eb21432ecba5981

In [7]:
my_print_number("e", e)


 e 

decimal: 293,245,277,964,256,583,618,315,291,376,862,756,400,904,631,956,766,353,666,145,261,926,942,488,480,954,370,418,946,574,008,925,975,481,860,302,549,733,879,553,875,478,496,332,312,974,952,326,509,321,090,086,493,119,050,743,283,873,016,181,065,305,205,061,711,321,627,128,638,611,547,186,739,471,403,935,400,540,006,504,582,018,140,042,048,817,703,755,731,862,625,803,083,397,623,561,704,475,389,586,416,219,220,497,951,414,894,583,301,336,165,560,725,014,606,806,932,492,667,034,923,156,238,392,335,223,038,106,534,805,688,137,123,567,792,161,411,681,241,709,527,855,646,978,678,688,494,643,236,463,332,370,631,139,695,320,818,283,265,196,036,505,177,519,455,369,663,556,134,147,388,978,817,628,912,852,213,776,612,052,838,043,572,663,844,864,673,824,539,452,066,354,128,575,024,978,171,982,029,108,491,769,770,681,009,671,025,172,169,301,915,446,002,018,375,128,049,984,794,054,888,769,078,826,149,561,118,463,507,823,562,743,356,936,343,676,778,659,522,201,599,066,941,671,238,641,6

In [8]:
# our first message with be 3
# we use the secret private key e to create a digital signature

m_1 = 3

digital_signature_1 = power_mod(m_1, e, n)

In [9]:
my_print_number("digital signature 1", digital_signature_1)


 digital signature 1 

decimal: 296,304,898,368,387,327,053,300,153,343,478,041,648,695,188,634,016,520,257,494,739,667,813,611,550,242,433,123,884,420,325,980,051,555,581,988,706,441,393,587,741,470,399,732,207,182,114,376,212,044,021,731,601,113,015,422,076,122,122,451,512,459,570,599,041,764,902,518,502,762,605,567,539,357,184,679,374,061,316,941,030,751,144,333,547,228,598,507,123,926,188,404,501,836,324,624,772,326,615,613,233,183,092,068,522,206,874,735,265,264,871,646,491,759,739,034,121,702,979,767,496,159,628,906,070,312,319,530,471,068,498,528,566,791,893,906,615,754,747,198,083,179,616,683,060,099,458,499,363,913,233,981,097,106,697,682,500,445,640,764,816,714,002,422,574,198,813,299,564,382,882,773,988,586,922,186,745,190,933,172,697,110,287,080,278,260,703,478,814,771,715,328,382,842,281,746,954,354,313,236,914,687,746,343,413,701,544,323,755,186,320,695,801,467,131,504,563,189,982,334,222,025,797,926,638,132,893,379,573,653,197,654,447,281,310,322,224,491,077,230,891,078

In [10]:
# verify our digital signature
# if we can decrypt it using the public key <d,n> and it matches 3, the digital signature is valid

v_1 = power_mod(digital_signature_1, d, n)

v_1

3

In [11]:
# our second message with be 5
# we use the secret private key e to create a digital signature

m_2 = 5

digital_signature_2 = power_mod(m_2, e, n)

In [12]:
my_print_number("digital signature 2", digital_signature_2)


 digital signature 2 

decimal: 258,683,223,627,496,718,100,253,376,641,196,807,843,056,632,276,330,199,964,140,760,267,516,844,339,892,362,702,506,971,839,151,655,509,675,687,935,753,204,052,151,155,901,680,346,870,121,918,922,634,298,235,461,321,933,012,789,753,929,390,911,354,233,425,475,563,764,566,289,754,173,630,948,942,635,047,648,964,713,027,678,060,132,431,641,847,944,780,996,746,246,612,481,974,060,292,876,616,421,864,910,917,135,357,268,964,909,063,446,580,000,912,800,900,850,497,997,163,900,086,522,844,506,496,782,257,532,111,844,981,850,465,297,953,700,130,237,370,988,284,558,247,808,850,488,452,050,728,275,044,948,281,455,493,229,008,879,915,752,986,321,857,914,324,265,482,038,304,596,278,374,854,137,878,355,709,897,935,415,392,032,026,013,785,025,875,166,266,263,397,556,032,496,772,925,318,549,809,928,776,223,588,410,413,792,550,369,279,233,701,132,683,091,949,444,940,387,080,115,002,310,134,349,088,780,932,957,346,794,567,035,780,232,467,353,852,086,438,092,777,214,975

In [13]:
# verify our digital signature
# if we can decrypt it using the public key <n,d> and it matches 5, the digital signature is valid

v_2 = power_mod(digital_signature_2, d, n)

v_2

5

In [14]:
# we will now forge a digital signature by multiplying the two digital signatures 

forged_digital_signature = (digital_signature_1 * digital_signature_2) % n

my_print_number("forged digital signature", forged_digital_signature)


 forged digital signature 

decimal: 496,589,147,383,990,695,004,463,811,866,087,792,349,621,788,315,986,104,166,621,662,587,937,071,596,857,159,146,821,935,091,935,965,499,125,013,038,118,926,016,442,130,935,647,406,832,227,135,991,772,173,931,650,333,629,415,910,840,379,025,731,907,030,004,258,596,258,613,706,262,994,498,221,863,467,476,129,425,658,688,254,449,004,079,960,440,618,284,194,882,265,879,841,529,083,235,957,826,951,992,166,229,885,376,906,520,799,275,890,264,276,405,619,997,429,178,422,531,626,528,866,781,577,676,582,479,424,992,265,505,552,477,618,415,900,417,746,304,100,080,957,045,914,700,925,707,024,079,519,921,088,752,021,612,173,020,954,617,882,341,220,585,415,910,109,178,101,110,870,347,433,729,859,513,113,436,466,427,404,490,336,529,836,713,296,028,689,023,531,408,975,216,711,666,531,076,848,505,291,358,432,724,276,314,981,800,716,848,198,761,168,270,812,764,739,495,518,436,904,891,093,812,434,959,706,522,176,260,427,162,711,047,911,520,629,555,191,080,532,853,15

In [15]:
# verify our forged digital signature
# if we can decrypt it using the public key <n,d> and it matches 15 (the product of 3 * 5), the digital signature is valid

v_3 = power_mod(forged_digital_signature, d, n)

v_3

15

### Add a simple padd to prevent attacks on RSA digital signatures using the homomorphic property, however, we will see that this introduces a pattern that could allow RSA to be broken

In [16]:
# to prevent attacks on RSA digital signatures using the homomorphic property, 
# we will convert the message to a string, add a 1 as a new least significant digit, 
# and then convert back to an integer

# our first message will again be 3

m_1 = 3

pad_1 = Integer(str(m_1) + "1")

digital_signature_pad_1 = power_mod(pad_1, e, n)


In [17]:
my_print_number("digital signature padded 1", digital_signature_pad_1)


 digital signature padded 1 

decimal: 80,458,444,498,570,345,657,724,143,993,055,394,951,434,717,643,406,500,997,973,824,151,792,950,609,158,952,731,225,292,779,522,062,709,590,338,157,401,471,986,836,387,676,876,755,231,408,867,828,269,287,036,778,806,314,630,916,262,150,506,703,798,404,470,603,593,657,635,264,371,458,276,412,217,291,245,935,450,695,556,378,284,002,158,245,427,296,344,509,660,684,342,596,568,263,274,253,119,827,088,860,724,644,320,296,159,669,130,808,751,149,289,431,351,921,962,365,172,206,331,668,907,036,678,539,348,454,701,034,810,824,260,838,560,006,423,427,269,571,195,326,282,235,397,698,618,491,890,024,789,320,379,465,315,507,314,441,308,941,740,097,908,479,763,425,018,460,410,926,441,031,349,611,433,368,847,522,437,824,190,003,449,883,714,201,011,780,832,342,054,670,495,112,258,054,184,401,512,038,519,436,610,757,862,376,804,823,843,698,221,296,152,885,047,628,842,274,278,791,123,615,992,023,904,910,773,206,921,051,270,383,753,433,519,383,221,602,876,516,166,7

In [18]:
# verify our padded digital signature
# if we can decrypt it using the public key <d,n>, remove the padding, and it matches 3, the digital signature is valid

v_1 = power_mod(digital_signature_pad_1, d, n)

v_unpad_1 = Integer(str(v_1)[:-1])

v_unpad_1

3

In [19]:
# to prevent attacks on RSA digital signatures using the homomorphic property, 
# we will convert the message to a string, add a 1 as a new least significant digit, 
# and then convert back to an integer

# our second message will again be 5

m_2 = 5

pad_2 = Integer(str(m_2) + "1")

digital_signature_pad_2 = power_mod(pad_2, e, n)


In [20]:
my_print_number("digital signature padded 2", digital_signature_pad_2)


 digital signature padded 2 

decimal: 386,692,511,829,789,649,710,373,685,824,792,570,452,985,664,325,411,508,126,630,184,895,025,227,050,678,161,845,475,428,886,226,052,472,893,780,312,638,822,012,291,002,706,513,747,010,449,518,019,599,265,879,395,003,069,772,885,022,827,952,284,129,216,954,163,475,580,417,134,384,782,906,115,462,823,768,692,366,301,937,184,258,914,122,280,515,685,840,732,024,472,380,143,252,634,797,530,234,786,216,091,627,769,036,824,072,209,225,526,951,861,723,498,758,965,602,218,817,883,647,465,674,783,415,561,371,269,906,769,566,261,790,823,043,223,247,361,461,827,551,486,525,484,398,001,225,399,995,810,925,066,138,176,717,615,901,510,634,891,817,645,599,485,354,719,137,443,660,040,436,014,726,908,302,706,081,726,039,045,277,630,831,874,849,706,714,699,449,969,462,770,228,279,148,299,380,498,885,931,718,350,311,963,769,465,080,448,838,986,728,462,182,146,875,017,593,008,943,133,203,606,141,676,160,181,776,853,006,973,225,401,882,927,491,747,788,486,224,334,334,

In [21]:
# verify our padded digital signature
# if we can decrypt it using the public key <d,n>, remove the padding, and it matches 5, the digital signature is valid

v_2 = power_mod(digital_signature_pad_2, d, n)

v_unpad_2 = Integer(str(v_2)[:-1])

v_unpad_2

5

In [22]:
# we will now attempt to forge a digital signature by multiplying the two digital signatures
# because of the padding to both digital signatures, it won't work

forged_digital_signature_pad = (digital_signature_pad_1 * digital_signature_pad_2) % n

my_print_number("forged digital signature padded", forged_digital_signature_pad)


 forged digital signature padded 

decimal: 104,110,835,075,391,867,990,935,619,946,171,060,527,648,175,469,522,458,806,589,749,081,634,341,480,941,185,240,290,495,309,971,783,424,987,915,753,082,763,931,856,206,486,784,522,263,985,638,870,476,880,371,163,277,697,830,001,857,736,537,880,596,653,204,470,687,675,292,881,543,153,179,843,156,539,980,799,339,176,161,673,214,897,486,683,469,057,923,805,062,162,895,135,807,988,183,714,395,160,516,149,427,552,991,112,498,937,053,387,585,665,842,487,613,395,109,133,346,890,930,028,411,965,245,796,303,502,014,864,003,123,399,272,716,531,033,817,438,653,923,584,288,121,871,299,714,630,941,057,259,432,267,656,612,080,533,956,261,459,862,116,347,333,065,732,310,900,294,980,134,226,187,781,173,574,891,901,638,867,333,468,719,146,476,245,925,695,593,064,592,866,436,328,809,823,311,698,882,624,686,568,470,378,404,909,420,289,777,077,788,429,191,890,800,009,095,353,874,383,923,878,267,704,881,291,227,216,769,436,584,577,352,590,263,587,921,861,787,159

In [23]:
# verify our forged digital signature with padding does not work
# if we can decrypt it using the public key <n,d> and it matches 15 (the product of 3 * 5), the digital signature is valid
# if it's another number (such as 158), we know the padding blocked attacks on the homomorphic property of RSA

v_3 = power_mod(forged_digital_signature_pad, d, n)

v_unpad_3 = Integer(str(v_3)[:-1])

v_unpad_3

158

### Add a random number padd of fixed length which will both prevent attacks on RSA using the homomorphic property without introducing easy patterns 

In [24]:
# in the last example, we added a single 1 as the new least significant digit
# this blocks attacks on the homomorphic property of RSA, 
# but it introduces a simple pattern that may be exploited in other attacks (what are the first two digits of 158?)
# one solution is to a add set number of digits from a random number as the new least significant digits
# each message would get a different pad, which destroys patterns in the padding


# use a different random pad each time, of a fixed number of digits (we will use 5), make them new least significant digits

# our first message will again be 3

m_1 = 3

random_pad = str(randint(10000,99999))

print ("random pad = ", random_pad)

pad_1 = Integer(str(m_1) + random_pad)

digital_signature_pad_1 = power_mod(pad_1, e, n)


random pad =  62700


In [25]:
my_print_number("digital signature padded 1", digital_signature_pad_1)


 digital signature padded 1 

decimal: 1,008,724,156,192,475,138,038,307,998,297,839,250,845,281,590,556,687,348,972,730,540,981,338,858,788,216,443,616,162,877,226,669,399,534,695,539,701,894,558,420,080,616,895,172,764,290,059,638,786,813,843,456,017,768,649,528,911,494,145,932,580,827,010,497,894,128,980,295,565,291,299,979,605,719,315,006,816,441,228,133,897,068,304,134,877,663,685,049,811,611,175,447,056,515,607,663,762,887,237,582,056,850,014,576,310,078,391,786,078,068,540,356,578,828,024,831,407,697,756,994,965,848,078,838,093,709,786,809,921,347,757,737,820,116,033,020,096,021,118,986,351,377,384,809,353,094,077,990,118,561,304,798,414,995,370,153,315,073,999,273,769,867,640,726,287,289,803,677,215,728,942,707,526,748,069,982,304,505,536,606,003,939,881,133,626,024,546,749,523,320,092,801,918,014,067,607,648,675,969,296,113,111,293,025,304,297,367,373,234,005,569,686,428,702,765,275,989,305,088,974,523,805,863,362,324,512,232,975,780,251,186,934,174,102,054,263,527,284,020,87

In [26]:
# verify our padded digital signature
# if we can decrypt it using the public key <d,n>, remove the padding, and it matches 3, the digital signature is valid

v_1 = power_mod(digital_signature_pad_1, d, n)

v_unpad_1 = Integer(str(v_1)[:-5])

v_unpad_1

3

In [27]:
# use a different random pad each time, of a fixed number of digits (we will use 5), make them new least significant digits

# our second message will again be 5

m_2 = 5

random_pad = str(randint(10000,99999))

print ("random pad = ", random_pad)

pad_2 = Integer(str(m_2) + random_pad)

digital_signature_pad_2 = power_mod(pad_2, e, n)

random pad =  82859


In [28]:
my_print_number("digital signature padded 2", digital_signature_pad_2)


 digital signature padded 2 

decimal: 23,403,173,208,524,247,594,371,345,715,248,641,331,659,901,906,794,219,981,166,553,744,909,879,710,220,467,429,249,440,593,782,243,200,871,497,119,039,300,036,495,852,159,493,524,241,411,472,787,805,242,199,564,855,635,112,261,973,611,939,137,740,548,981,935,757,412,307,021,446,889,949,513,228,163,661,427,438,205,397,974,340,135,156,773,568,412,330,753,500,589,715,756,187,924,461,316,715,403,148,617,878,194,484,161,474,372,972,952,502,534,408,301,241,276,933,762,375,919,538,702,472,645,549,389,466,952,476,799,372,969,625,317,328,907,659,017,613,643,522,135,683,825,741,853,792,987,216,236,235,211,329,458,773,011,329,380,897,119,888,380,936,921,425,547,772,355,313,300,593,987,785,989,279,914,187,676,250,583,088,579,121,093,185,011,273,793,974,753,478,679,804,862,176,719,908,681,423,988,006,751,537,987,525,308,305,702,017,617,220,660,018,161,996,742,425,570,167,370,710,167,061,671,450,073,680,516,764,778,190,619,601,846,796,363,886,662,430,857,471,4

In [29]:
# verify our padded digital signature
# if we can decrypt it using the public key <d,n>, remove the padding, and it matches 5, the digital signature is valid

v_2 = power_mod(digital_signature_pad_2, d, n)

v_unpad_2 = Integer(str(v_2)[:-5])

v_unpad_2

5

In [30]:
# we will now attempt to forge a digital signature by multiplying the two digital signatures
# because of the padding to both digital signatures, it won't work

forged_digital_signature_pad = (digital_signature_pad_1 * digital_signature_pad_2) % n

my_print_number("forged digital signature padded", forged_digital_signature_pad)


 forged digital signature padded 

decimal: 177,928,909,270,214,295,160,241,027,185,026,214,939,146,119,257,900,973,725,341,744,311,283,760,350,669,725,451,710,402,911,223,619,585,988,567,819,718,323,515,652,223,091,080,257,360,317,014,925,318,925,358,422,070,676,964,864,477,105,608,544,821,888,004,573,880,277,932,311,371,665,809,922,076,038,159,265,623,693,972,871,341,856,604,626,591,688,834,584,087,603,620,844,041,728,730,229,457,255,447,862,804,580,818,235,693,351,675,764,970,040,622,701,635,314,937,288,466,290,183,094,031,328,944,396,084,890,066,786,112,472,628,735,170,644,984,667,888,421,958,380,055,611,769,969,272,837,469,670,895,832,141,020,540,114,194,577,390,345,715,335,173,811,738,007,125,935,208,964,055,439,472,605,904,389,802,832,938,959,193,958,823,527,856,642,578,528,710,302,329,285,617,572,368,690,197,326,729,245,074,303,924,929,227,306,676,041,492,385,043,072,785,057,632,346,319,174,846,802,477,732,787,088,332,285,462,903,486,574,383,348,727,990,821,170,218,279,869,312

In [31]:
# verify our forged digital signature with padding does not work
# if we can decrypt it using the public key <n,d> and it matches 15 (the product of 3 * 5), the digital signature is valid
# if it's another number, we know the padding blocked attacks on the homomorphic property of RSA

v_3 = power_mod(forged_digital_signature_pad, d, n)
x = str(v_3)[:-5]

if len(x) > 0:
    v_unpad_3 = Integer(x)
else:
    v_unpad_3 = float('-inf')

v_unpad_3

2114029

Note that if we re-run the padding of a single 1, we get the same result every time (158)

Note that if we re-run the padding using a random number, we get a different result every time which shows that patterns have been broken.